In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.api import Holt
from sklearn.metrics import mean_squared_error

from prophet import Prophet

plt.rcParams["figure.figsize"] = (14, 8)
import wrangle

ModuleNotFoundError: No module named 'prophet'

# Acquisition, prep, and split

In [ ]:
# Pulling in all data, cleaning and displaying info
aquifer, temps, precip, pop, usage = wrangle.get_dataframes()
aquifer, weather, pop, usage = wrangle.clean_all_dataframes(aquifer, temps, precip, pop, usage)
aquifer.info(), weather.info(), pop.info(), usage.info()

In [ ]:
# Setting train size to be 60% of the total dataset
train_size = int(round(aquifer.shape[0] * 0.6))

# set validate size to be 25% of the total dataset
validate_size = int(round(aquifer.shape[0] * 0.25))

# Setting test size to be 15% of the total dataset. 
test_size = int(round(aquifer.shape[0] * 0.15))

In [ ]:
# Checking to make sure the split worked
len(aquifer) == train_size + validate_size + test_size

In [ ]:
validate_end_index = train_size + validate_size
validate_end_index

In [ ]:
train = aquifer[:train_size]
validate = aquifer[train_size:validate_end_index]
test = aquifer[validate_end_index:]

In [ ]:
# Checking the split to see if the starting and ending values for train, validate and test line up properly.
train.tail(1), validate.head(1), validate.tail(1), test.head(1)

In [ ]:
#Plotting out the splits to visualize them
plt.plot(train)
plt.plot(validate)
plt.plot(test)
plt.show()

In [ ]:
# Making sure the datasets have accurate frequency data
train = train.asfreq('d', method='bfill')
validate = validate.asfreq('d', method='bfill')
test = test.asfreq('d', method='bfill')

In [ ]:
def evaluate(target_var):
    '''
    This function takes in the actual values of the target_var from validate, and the predicted values stored in yhat_df, 
    and computes the rmse, rounding to 0 decimal places. Finally it returns the rmse. 
    '''
    rmse = round((mean_squared_error(validate[target_var], yhat_df[target_var]))**(1/2), 0)
    return rmse

In [ ]:
def plot_and_eval(target_var):
    '''
    This function takes in the target var name (string), and returns a plot
    of the values of train for that variable, validate, and the predicted values from yhat_df. 
    it will als lable the rmse. 
    '''
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label='Train', linewidth=1)
    plt.plot(validate[target_var], label='Validate', linewidth=1)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

In [ ]:
# create an empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

In [ ]:
# function to store the rmse so that we can compare
def append_eval_df(model_type, target_var):
    '''
    this function takes in as arguments the type of model run, and the name of the target variable. 
    It returns the eval_df with the rmse appended to it for that model and target_var. 
    '''
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var],
        'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

# Modeling, Forecasting, Predicting

For modeling I will start with some basic predictions in order to acquire a baseline.
1. Last Observed Value: This will predict that future values will look like the most recent value.
2. Simple Average: This will predict that future values will look like a historical average. 
3. Moving Average: This will predict that future values will look like a recent period of time's average. 

Then I will move into more predictive models: 

4. Holt's Linear Trend
5. Previous Cycle

And finally if given time I want to try either:

6. Facebook Prophet
7. LinkedIn Greykite

### Last observed value

The simplest method for forecasting is to predict all future values to be the last observed value.  

In [ ]:
# Grabbing the most recent observation in train and assigning it to a variable
last_observed_level = train['water_level_elevation'][-1:][0]
last_observed_level

In [ ]:
yhat_df = pd.DataFrame({'water_level_elevation': [last_observed_level]}, index=validate.index)
yhat_df.head()

In [ ]:
plot_and_eval('water_level_elevation')

In [ ]:
eval_df = append_eval_df(model_type = 'last_observed_value', target_var = 'water_level_elevation')
eval_df

### Simple Average

This takes the historical average and uses that to predict future values.   

This is a possible good option for an initial baseline.

In [ ]:
# Computing the historical mean
avg_elevation = round(train['water_level_elevation'].mean(), 2)
avg_elevation

In [ ]:
yhat_df = pd.DataFrame({'water_level_elevation': [avg_elevation]}, index=validate.index)
yhat_df.head()

In [ ]:
plot_and_eval('water_level_elevation')

In [ ]:
eval_df = append_eval_df(model_type = 'avg_elevation', target_var = 'water_level_elevation')
eval_df

### Moving Average

I will use a range of different rolling averages to see which works the best.

In [ ]:
periods = [7, 14, 30, 60, 365, 730, 1825, 3650]

for p in periods: 
    rolling_water_levels = round(train['water_level_elevation'].rolling(p).mean()[-1], 2)
    yhat_df = pd.DataFrame({'water_level_elevation': [rolling_water_levels]},
                          index=validate.index)
    model_type = str(p) + '_day_moving_avg'
    for col in train.columns:
        eval_df = append_eval_df(model_type = model_type,
                                target_var = col)

In [ ]:
eval_df

The best performing model so far is the 730 day (2 year) moving avg with an RMSE of 17 although the overall average isn't farm behind with an RMSE of 18. So going forward the 730_day_moving_avg will be my baseline with RMSE of 17.

### Holt's Linear Trend

For the initial Holt's model I'm going to use base settings and tweak hyperparameters later if needed.

In [ ]:
# Creating the initial Holt's Object
model = Holt(train, exponential=False, damped=True)
# Fitting the Holt's object
model = model.fit(optimized=True)
#Making predictions for each date in validate
yhat_items = model.predict(start = validate.index[0], end = validate.index[-1])

In [ ]:
# Checking to make sure the length of validate matches the number of predictions given in my yhat_tems
validate.shape, yhat_items

In [ ]:
# add predictions to yhat_df
yhat_df['water_level_elevation'] = pd.DataFrame(yhat_items)

In [ ]:
yhat_df

In [ ]:
plot_and_eval('water_level_elevation')

In [ ]:
eval_df = append_eval_df(model_type = 'holts_optimized', target_var = 'water_level_elevation')
eval_df

Holt's linear trend model performed terribly. Since there is no identifiable previous cycle, I can't use this method to predict future values.

I will need to find a better model if I am going to beat baseline, but it's entirely possible there is no model that will accurately predict this based on water level elevation data alone since it's seemingly random. Thus univariate time series forecasts would not be possible. Taking into account other variable data could would be valuable but as it stands this is beyond the scope of this project.